<a href="https://colab.research.google.com/github/Adronz/Fungal-Proteomics/blob/main/Copy_of_Fungi_Clustering.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
import sys
import gzip
import numpy as np
import pandas as pd
import urllib.request
!pip install umap-learn
import umap.umap_ as umap
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder
from matplotlib.colors import ListedColormap
from matplotlib.lines import Line2D

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.7/85.7 kB 2.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.8/56.8 kB 6.3 MB/s eta 0:00:00


In [ ]:
#!/usr/bin/python3

import os
import sys
import gzip
import numpy as np
import pandas as pd

txipIn= "txip_fungi.npy"
txidxIn= "ipromerge_fungi.txt";
ipidxIn= "ipromerge_fungi.txt";

# interpro domain info
# https://ftp.ebi.ac.uk/pub/databases/interpro/current_release/entry.list

# tax id info ( ... with gzip.open(file_name, "rt") as ... )
#   proteomes_AND_proteome_type_1_AND_super_2023_09_20.tsv.gz
#   proteomes_AND_superkingdom_Bacteria_AND_2023_09_20.tsv.gz
#   proteomes_AND_Eucaryota_AND_super_2023_11_01.tsv.gz

txid = []
txidx = {}

ipid =[]
ipidx = {}

with open(txidxIn,"r") as fh:
    for ln in fh:
        if ln.startswith("tx:"):
            col = ln.strip().split()
            txid.append( col[1].strip() )
            txidx[ col[1].strip() ] = len(txid)-1

with open(ipidxIn,"r") as fh:
    for ln in fh:
        if ln.startswith("ipd:"):
            col = ln.strip().split()
            ipid.append(col[1].strip())
            ipidx[ col[1].strip() ] = len(ipid)-1

print( "tax:", len(txid), txid[:10] )
print( "dom:", len(ipid), ipid[:10] )

# numpy array
#------------

txip = np.load( txipIn )

print( txip.shape )
print( txip )
print( txip[:10,:10] )


# pandas dataframe
#-----------------

tipd = pd.DataFrame( txip, index= txidx, columns=ipidx )
print(tipd.shape)
print(tipd)

FileNotFoundError: [Errno 2] No such file or directory: 'ipromerge_fungi.txt'

In [ ]:
# TODO: not sure if this entry_list is right
entry_list = pd.read_csv(r'entry_list.txt', sep = '\t')
entry_list.head()

In [ ]:
with gzip.open("proteomes_AND_Eucaryota_AND_super_2023_11_01.tsv.gz", 'r') as f:
    string_data = f.read().decode()
    list_data = [row.split("\t") for row in string_data.split("\n")]
    df_entries_eukaryota = pd.DataFrame(list_data[1:], columns=list_data[0])
#create df for all eukaryotes
#organism_organismid_df_archea = df_entries_archea.iloc[:, 1:3]
organism_organismid_df_eukaryota = pd.concat([df_entries_eukaryota['Organism Id'], df_entries_eukaryota['Organism']], axis=1)
df_entries_eukaryota.head()

In [ ]:
df_entries_eukaryota.shape

In [ ]:
#get ids for all eukaryota in original table from tipd
row_names = list(tipd.index.values.tolist())

eukaryota_ids = df_entries_eukaryota['Organism Id']
eukaryota_ids = eukaryota_ids.to_list()
subset_ids_eukaryota = []
eukaryota = []
for i in range(len(row_names)):
    if row_names[i] in eukaryota_ids:
        subset_ids_eukaryota.append(i)
        eukaryota.append("Eukaryota")
    else: eukaryota.append("N/A")


print(eukaryota_ids)


In [ ]:
print(None in eukaryota_ids)

In [ ]:
len(eukaryota_ids)

In [ ]:
'''
with gzip.open("proteomes_AND_Eucaryota_AND_super_2023_11_01.tsv.gz", 'r') as f:
    string_data = f.read().decode()
    list_data = [row.split("\t") for row in string_data.split("\n")]
    df_entries_eukaryota = pd.DataFrame(list_data[1:], columns=list_data[0])
df_entries_eukaryota.head()
'''

In [ ]:
#global organism_organismid
organism_organismid = pd.DataFrame(organism_organismid_df_eukaryota)
organism_organismid.head()

In [ ]:
organism_organismid.shape

In [ ]:
'''
#get ids for all eukaryota
row_names = list(tipd.index.values.tolist())
print(row_names)

eukaryota_ids = df_entries_eukaryota['Organism Id']
eukrayota_ids = eukrayota_ids.to_list()
# print(bacteria_ids)
# print(len(row_names))
subset_ids_eukrayota = []
eukaryota = []
for i in range(len(row_names)):
    if row_names[i] in eukrayota_ids:
        subset_ids_eukaryota.append(i)
        eukrayota[i] = "Eukaryota"

# print(subset_ids_bacteria)
# print(bacteria_archea)
# print(len(bacteria_ids))

# alr did except would add "N/A" if not found
'''

In [ ]:
'''
tipd['Eukaryota'] = eukaryota
tipd.head()
'''

In [ ]:
eukaryota_ids = [str(i) for i in df_entries_eukaryota['Organism Id'].to_list()]
eukaryota_ids

In [ ]:
def extract_taxonomic_ranks(df, levels=5):
    # Split the 'Taxonomic lineage' by ', ' and expand to the desired number of columns
    tax_ranks = df['Taxonomic lineage'].str.split(', ', expand=True)

    # Ensure there are enough columns, filling with NA if necessary
    for _ in range(tax_ranks.shape[1], levels):
        tax_ranks[_] = np.nan  # Add a new column filled with NaNs

    # Take only the first `levels` columns
    tax_ranks = tax_ranks.iloc[:, :levels]

    # Name the columns appropriately
    tax_ranks.columns = ['Domain', 'Phylum', 'Class', 'Order', 'Family']

    # Replace empty strings with NaN
    tax_ranks.replace('', np.nan, inplace=True)

    # Set the "Organism Id" as the index
    tax_ranks.index = df['Organism Id']

    return tax_ranks

archaea_tax_ranks = extract_taxonomic_ranks(df_entries_eukaryota)
bacteria_tax_ranks = extract_taxonomic_ranks(df_entries_eukaryota)

df_combined = pd.concat([archaea_tax_ranks, bacteria_tax_ranks], ignore_index=False)
df_combined = df_combined[~df_combined.index.isna()]
df_combined = df_combined[df_combined.index.isin(tipd.index)]
df_combined

In [ ]:
def eukaryota_tax_lineage(level, df_entries_eukaryota):
  eukaryota_taxonomic_info = df_entries_eukaryota['Taxonomic lineage'].str.split(', ')
  eukaryota_phylum = eukaryota_taxonomic_info.str[level]
  df_entries_eukaryota['Taxonomic lineage'] = eukaryota_phylum

  # Extract Phylum and form a list
  eukaryota_tax_lineage = df_entries_eukaryota['Taxonomic lineage'].to_list()

  # Create a dictionary with Oragnism Id as keys and Phylum as values
  eukaryota_tax_lineage = pd.Series(eukaryota_tax_lineage, index = eukaryota_ids).to_dict()
  print(eukaryota_tax_lineage)
  return eukaryota_tax_lineage, df_entries_eukaryota

#archaea_tax_lineage, df_entries_archea = archaea_tax_lineage(1, df_entries

In [ ]:
'''
bacteria_ids = [str(i) for i in df_entries_bacteria['Organism Id'].to_list()]
def bacteria_tax_lineage(level, df_entries_bacteria):
  bacteria_taxonomic_info = df_entries_bacteria['Taxonomic lineage'].str.split(', ')
  bacteria_phylum = bacteria_taxonomic_info.str[1]
  df_entries_bacteria['Taxonomic lineage'] = bacteria_phylum
  bacteria_tax_lineage = df_entries_bacteria['Taxonomic lineage'].to_list()
  bacteria_tax_lineage = pd.Series(bacteria_tax_lineage, index = bacteria_ids).to_dict()
  print(bacteria_tax_lineage)
  return bacteria_tax_lineage, df_entries_bacteria

bacteria_tax_lineage, df_entries_bacteria = bacteria_tax_lineage(1, df_entries_bacteria)
'''

In [ ]:
eukaryota_tax_lineage, df_entries_eukaryota = eukaryota_tax_lineage(1, df_entries_eukaryota)
tax_id = list(tipd.index.values.tolist())

new_id = []
tax_lineage = []
eukaryota_lineage = []

# Go through each tax_id and match the corresponding phylum information
for i in range(len(tax_id)):
    if tax_id[i] in eukaryota_ids:
        new_id.append(tax_id[i])
        tax_lineage.append(eukaryota_tax_lineage[tax_id[i]])
        eukaryota_lineage.append(eukaryota_tax_lineage[tax_id[i]])
    else:
        print(f"Tax ID {tax_id[i]} not found in eukaryota_ids")



In [ ]:
result_df = pd.DataFrame({'Organism Id': new_id, 'Taxonomic lineage': tax_lineage})
tax_lineage_unique = list(set(tax_lineage))
tax_lineage_eukaryota_unique = list(set(eukaryota_lineage))
print(tax_lineage_eukaryota_unique)
result_df

In [ ]:
tipd['Taxonomic lineage'] = tax_lineage
tipd_reset_index = tipd.reset_index()

In [ ]:
tipd_reset_index

In [ ]:
!pip install plotly
#==5.10
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from plotly import express as px
import plotly.io as pio
print("Done")

In [ ]:
reducer = umap.UMAP()
tax_level=["Phylum", "Class", "Order", "Family"]

##tipd columns are IPR numbers
# print("columns")
# for col in tipd.columns:
#     print(col)

#dropdown_widget = [widgets.Dropdown(options=tax_level, value='Class', description='Taxonomic Rank:', disabled=False)]
#checkboxes_tax_level=[widgets.Checkboxes(value=True, description=label) for label in tax_level]
#ui = widgets.VBox(dropdown_widget)
try:
    tipd = tipd.drop('Eukaryota', axis=1)
except KeyError:
    pass
embedding = reducer.fit_transform(tipd.loc[:, tipd.columns != 'Taxonomic lineage'])

checkboxes = [widgets.Checkbox(value=True, description=label) for label in tax_lineage_unique]
ui = widgets.VBox(children=checkboxes)

def get_selected_data(checkboxes):
  selected_data = []
  for i in range(0, len(checkboxes)):
      if checkboxes[i].value == True:
          selected_data = selected_data + [checkboxes[i].description]

In [ ]:
def update_plot(**kwargs):
  selected_data = []
  for k, v in kwargs.items():
    if v==True:
      selected_data.append(k)

  tipd_bool_temp = tipd['Taxonomic lineage'].isin(selected_data)
  tax_lin_temp = tipd[tipd_bool_temp]
  tipd_temp = tipd.index[tipd_bool_temp]

  global tipd_reset_index
  indexes_list = tipd_temp.tolist()
  #tipd_reset_index = tipd.reset_index() #this line is taking a long time
  reset_index_list = tipd_reset_index.index[tipd_reset_index['Taxonomic lineage'].isin(selected_data)]
  organism_id = [str(i) for i in indexes_list]


  global organism_organismid
  organism_organismid_temp = organism_organismid[organism_organismid['Organism Id'].isin(organism_id)]
  organism_organismid_temp = organism_organismid_temp.set_index('Organism Id').loc[organism_id].reset_index()

  embedding_temp = embedding[reset_index_list]

  pio.templates.default = "plotly_white"
  print("before scattering")
  fig = px.scatter(x = embedding_temp[:, 0],
                  y = embedding_temp[:, 1],
                  color = tax_lin_temp['Taxonomic lineage'],
                  hover_name = organism_organismid_temp['Organism'],
                  title = "Fungi Taxonomic Lineage by Class",
                  width = 800,
                  height = 500,
                  opacity = 0.5,)
  fig.update_layout(margin = {"r": 0, "t": 50, "l": 0, "b": 0})
  print("before showing")
  fig.show()
  print("after showing")

interact(update_plot, **{c.description:c.value for c in checkboxes})

In [ ]:
# K-Means clustering
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import make_blobs
from sklearn.cluster import KMeans
import umap

In [ ]:
kmeans = KMeans(n_clusters=15)
kmeans.fit(embedding)

In [ ]:
plt.scatter(embedding[:, 0], embedding[:, 1], c=kmeans.labels_, cmap='tab20', s=5)
centers = kmeans.cluster_centers_
# plt.scatter(centers[:, 0], centers[:, 1], c='red', marker='X', s=100, alpha=0.5)
plt.title('Fungi UMAP + K-means Clustering')
plt.colorbar(label='Cluster')
plt.show()

In [ ]:
cluster_labels = kmeans.labels_
cluster_labels

In [ ]:
cluster_percentage = {}
for cluster_label in np.unique(cluster_labels):
    cluster_indices = np.where(cluster_labels == cluster_label)[0]
    cluster_data = tipd.iloc[cluster_indices]
    total_samples_in_cluster = len(cluster_data)
    feature_counts = cluster_data['Taxonomic lineage'].value_counts()
    feature_percentages = (feature_counts / total_samples_in_cluster) * 100
    cluster_percentage[cluster_label] = feature_percentages

# Step 4: Create a table to display percentages
table_data = []
for cluster_label, percentages in cluster_percentage.items():
    for feature_value, percentage in percentages.items():
        table_data.append([cluster_label, feature_value, percentage])

table_columns = ['Cluster', 'Taxon', 'Percentage']
percentage_table = pd.DataFrame(table_data, columns=table_columns)

In [ ]:
percentage_table

In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# Assuming 'percentage_table' contains your DataFrame with cluster percentages

# Get unique clusters
clusters = percentage_table['Cluster'].unique()

# Define the layout of the grid
num_clusters = len(clusters)
num_cols = 3  # Number of columns in the grid
num_rows = int(np.ceil(num_clusters / num_cols))  # Number of rows in the grid

# Get unique feature values
unique_feature_values = percentage_table['Taxon'].unique()

# Define a list of colors for each feature value
colors = plt.cm.tab20c(np.linspace(0, 1, len(unique_feature_values)))

# Create a dictionary mapping feature values to colors
color_dict = dict(zip(unique_feature_values, colors))

# Create the grid of subplots
fig, axs = plt.subplots(num_rows, num_cols, figsize=(15, 5*num_rows))

# Plot pie chart for each cluster
for i, cluster in enumerate(clusters):
    cluster_data = percentage_table[percentage_table['Cluster'] == cluster]
    feature_values = cluster_data['Taxon']
    percentages = cluster_data['Percentage']

    # Get colors for each feature value
    feature_colors = [color_dict[value] for value in feature_values]

    row = i // num_cols
    col = i % num_cols

    ax = axs[row, col] if num_rows > 1 else axs[col]
    ax.pie(percentages, labels=feature_values, autopct='%1.1f%%', colors=feature_colors)
    ax.set_title(f'Cluster {cluster} - Taxon Distribution')

# Hide empty subplots if any
for i in range(len(clusters), num_rows*num_cols):
    row = i // num_cols
    col = i % num_cols
    axs[row, col].axis('off')

plt.tight_layout()
plt.show()


In [ ]:
import pandas as pd
import plotly.express as px

# Create a pandas dataframe with the taxonomic data
df = pd.DataFrame({
    "taxon": ["Animalia", "Chordata", "Mammalia", "Primates", "Hominidae", "Homo", "Homo sapiens"],
    "parent": ["None", "Animalia", "Chordata", "Mammalia", "Primates", "Hominidae", "Homo"],
    "value": [100, 90, 80, 70, 60, 50, 40]
})

# Create the sunburst chart
fig = px.sunburst(df, path="taxon", values="value")

# Update the layout
fig.update_layout(title="Taxonomic Sunburst Chart")

# Display the chart
fig.show()